In [2]:
import os
import sys
import logging

import pandas as pd

# import seaborn as sns
# sns.set_theme()
# sns.set_color_codes()
import matplotlib.pyplot as plt
import numpy as np

import plotly.express as px
import plotly.io as pio

pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go


# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
sys.path.append("../components/paddock/")

%load_ext autoreload
%autoreload 2

from telemetry.influx import Influx
from telemetry.analyzer import Analyzer

from IPython.display import Image

pd.set_option("display.max_columns", None)

influx = Influx()
influx.laps_from_file("iracing.csv")
analyzer = Analyzer()

In [3]:
(
    game,
    session,
    track,
    car,
    # ) = "iRacing,1674828783,barcelona gp,Ferrari 488 GT3 Evo 2020".split(",")      # circuit
    # ) = "iRacing,1674679321,phoenix rallycross,Volkswagen Beetle GRC ".split(",")  # rallycross
    # ) = "iRacing,1670307078,fiveflags,Chevrolet Monte Carlo SS".split(",")         # oval
    # ) = "iRacing,1673296770,nurburgring nordschleifetourist,Porsche 911 GT3 Cup (991)".split(",")         # long track
) = "iRacing,1676736012,limerock 2019 gp,Ferrari 488 GT3 Evo 2020".split(
    ","
)  # short track
all_laps = influx.telemetry_for(game=game, track=track, car=car)

In [4]:
lap = all_laps[0].copy()

# find the index where the lap starts, thats where CurrentLapTime is minimal
# only keep the part of the lap after the start
lap = lap[["id", "DistanceRoundTrack", "CurrentLapTime", "SpeedMs"]]
lap = analyzer.resample(lap, freq=1, columns=["CurrentLapTime", "SpeedMs"])
lap_start = lap["CurrentLapTime"].idxmin()

display(lap_start)

# for all indices until lap_start, calculate a new value for CurrentLapTime
# CurrentLapTime = DistanceRoundTrack * SpeedMs

lap.loc[:lap_start, "CurrentLapTime"] = (
    lap.loc[:lap_start, "DistanceRoundTrack"] / lap.loc[:lap_start, "SpeedMs"]
)
lap.loc[lap_start - 10 : lap_start + 10]

107

,DistanceRoundTrack,CurrentLapTime,SpeedMs,id
97,97.815591,1.441554,67.854269,1676736012-11
98,98.816100,1.455972,67.869504,1676736012-11
99,99.816610,1.470385,67.884666,1676736012-11
100,100.817119,1.484805,67.899215,1676736012-11
101,101.817628,1.499230,67.913274,1676736012-11
102,102.818137,1.513648,67.927380,1676736012-11
103,103.818647,1.528051,67.941873,1676736012-11
104,104.819156,1.542470,67.955407,1676736012-11
105,105.819665,1.556888,67.968716,1676736012-11
106,106.820174,1.571326,67.980910,1676736012-11


In [5]:
# remove id and SpeedMs column
lap = lap[["DistanceRoundTrack", "CurrentLapTime"]]
# round DistanceRoundTrack to 1 decimal
lap["DistanceRoundTrack"] = lap["DistanceRoundTrack"].round(1)
# round CurrentLapTime to 3 decimals
lap["CurrentLapTime"] = lap["CurrentLapTime"].round(3)
lap

,DistanceRoundTrack,CurrentLapTime
0,0.8,0.012
1,1.8,0.027
2,2.8,0.042
3,3.8,0.057
4,4.8,0.072
...,...,...
2332,2334.0,52.001
2333,2335.0,52.016
2334,2336.0,52.031
2335,2337.0,52.046


In [24]:
def offset_distance(distance_time, distance, seconds=0.0):
    lap_time = distance_time.loc[distance]["CurrentLapTime"]
    lap_time_at_offset = lap_time - seconds

    if lap_time_at_offset < 0:
        # FIXME wrap around and start counting back from the end of the track
        return 0

    # loop backwards until we find the first index where CurrentLapTime is smaller than offset
    while lap_time > lap_time_at_offset and distance > 0:
        distance -= 1
        lap_time = distance_time.loc[distance]["CurrentLapTime"]
    return distance


meters = 80
offset = 1.2
distance_time = offset_distance(lap, meters, seconds=offset)
display(distance_time)

80

In [38]:
df = lap.iloc[10:20]
display(df)
idx = df["DistanceRoundTrack"].sub(12).abs().idxmin()
df.loc[1]

,DistanceRoundTrack,CurrentLapTime
10,10.8,0.162
11,11.8,0.177
12,12.8,0.192
13,13.8,0.207
14,14.8,0.222
15,15.8,0.237
16,16.8,0.252
17,17.8,0.267
18,18.8,0.282
19,19.8,0.297


DistanceRoundTrack    11.800
CurrentLapTime         0.177
Name: 11, dtype: float64

In [49]:
df = pd.DataFrame()
t = pd.DataFrame(
    [
        {
            "a": 1,
            "b": 2,
        }
    ]
)

df = pd.concat([df, t], ignore_index=True)
df = pd.concat([df, t], ignore_index=True)
df = pd.concat([df, t], ignore_index=True)
df

,a,b
0,1,2
1,1,2
2,1,2
